In [ ]:
import os
import random
import shutil
from glob import glob
from google.colab import drive

# --- Configuration ---
drive.mount('/content/drive', force_remount=True)
gopro_base_path = "/content/drive/My Drive/gopro_deblur" # ADJUST THIS PATH
sharp_folder = os.path.join(gopro_base_path, "sharp/images")
blur_folder = os.path.join(gopro_base_path, "blur/images")

# --- Output Structure ---
# We'll create this structure in your Colab environment
output_base = "/content/GoPro_Split"
train_A_path = os.path.join(output_base, "train", "trainA") # Blurry for training
train_B_path = os.path.join(output_base, "train", "trainB") # Sharp for training
test_A_path = os.path.join(output_base, "test", "testA")   # Blurry for testing
test_B_path = os.path.join(output_base, "test", "testB")    # Sharp for testing

# --- Split Ratio ---
test_split_ratio = 0.1 # Use 10% for testing, 90% for training

# --- Create Output Directories ---
os.makedirs(train_A_path, exist_ok=True)
os.makedirs(train_B_path, exist_ok=True)
os.makedirs(test_A_path, exist_ok=True)
os.makedirs(test_B_path, exist_ok=True)

# --- Get Image Files ---
sharp_images = sorted(glob(os.path.join(sharp_folder, "*.*")))
blur_images = sorted(glob(os.path.join(blur_folder, "*.*")))

if not sharp_images or not blur_images:
    print(f"Error: No images found in {sharp_folder} or {blur_folder}. Check paths.")
elif len(sharp_images) != len(blur_images):
    print(f"Error: Mismatch in number of sharp ({len(sharp_images)}) and blur ({len(blur_images)}) images.")
else:
    print(f"Found {len(sharp_images)} paired images.")

    # --- Shuffle and Split ---
    paired_list = list(zip(sharp_images, blur_images))
    random.shuffle(paired_list) # Shuffle pairs together
    num_test = int(len(paired_list) * test_split_ratio)
    test_pairs = paired_list[:num_test]
    train_pairs = paired_list[num_test:]

    print(f"Splitting into {len(train_pairs)} training pairs and {len(test_pairs)} testing pairs.")

    # --- Copy Files ---
    def copy_pairs(pair_list, dest_A, dest_B):
        for sharp_path, blur_path in pair_list:
            base_name = os.path.basename(sharp_path) # Use sharp name for both
            try:
                shutil.copy(blur_path, os.path.join(dest_A, base_name))
                shutil.copy(sharp_path, os.path.join(dest_B, base_name))
            except Exception as e:
                print(f"Error copying {base_name}: {e}")

    print("Copying training files...")
    copy_pairs(train_pairs, train_A_path, train_B_path)
    print("Copying testing files...")
    copy_pairs(test_pairs, test_A_path, test_B_path)

    print("\nDataset splitting complete!")
    print(f"Training data in: {output_base}/train/")
    print(f"Testing data in: {output_base}/test/")

Mounted at /content/drive
Found 1029 paired images.
Splitting into 927 training pairs and 102 testing pairs.
Copying training files...
Copying testing files...

Dataset splitting complete!
Training data in: /content/GoPro_Split/train/
Testing data in: /content/GoPro_Split/test/


In [ ]:
!git clone https://github.com/AhsanBaidar/MuLA_GAN.git

Cloning into 'MuLA_GAN'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 125 (delta 30), reused 78 (delta 15), pack-reused 28 (from 1)
Receiving objects: 100% (125/125), 144.94 MiB | 38.65 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
%cd MuLA_GAN

/content/MuLA_GAN


In [ ]:
# --- Edit configs/train_MuLA-GAN.yaml ---
CONFIG_FILE="configs/train_MuLA-GAN.yaml"

# Set the ABSOLUTE path for the input training images
!sed -i "s|TRAIN_INPUT: '.*'|TRAIN_INPUT: '/content/GoPro_Split/train/trainA'|" $CONFIG_FILE

# Set the ABSOLUTE path for the ground truth training images
!sed -i "s|TRAIN_GT: '.*'|TRAIN_GT: '/content/GoPro_Split/train/trainB'|" $CONFIG_FILE

# The DATASET line might not even be needed now, but we'll leave it
# !sed -i "s|DATASET: '.*'|DATASET: '/content/GoPro_Split'|" $CONFIG_FILE # This line likely doesn't matter anymore

# Optional: Set batch size if needed
# !sed -i 's/BATCH_SIZE: .*/BATCH_SIZE: 4/' $CONFIG_FILE

print("--- Config file train_MuLA-GAN.yaml updated with ABSOLUTE paths ---")
!cat $CONFIG_FILE # Verify the changes

--- Config file train_MuLA-GAN.yaml updated with ABSOLUTE paths ---

# dataset info
dataset_name: "UIEB" 
dataset_path: "./Dataset/"

# image info
chans: 3
im_width: 256
im_height: 256

# other params
val_interval: 1000 # steps
ckpt_interval: 20  # epochs



 


In [ ]:
%%writefile utils/data_utils.py
import os
import glob
import random
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import yaml # Need yaml to read the config inside the function

class Dataloader(Dataset):
    def __init__(self, root, dataset_name, transforms_=None, config_path="configs/train_MuLA-GAN.yaml"): # Added config_path
        self.transform = transforms.Compose(transforms_)

        # --- Read config to get correct paths ---
        try:
            with open(config_path) as f:
                cfg = yaml.load(f, Loader=yaml.FullLoader)
            input_path = cfg["TRAIN_INPUT"]
            gt_path = cfg["TRAIN_GT"]
            # Check if paths in config are relative or absolute
            # If relative, join them with the root path from the main script
            # If absolute, use them directly
            if not os.path.isabs(input_path):
                 input_path = os.path.join(root, input_path)
            if not os.path.isabs(gt_path):
                 gt_path = os.path.join(root, gt_path)

        except Exception as e:
            print(f"Error reading config file {config_path} in Dataloader: {e}")
            # Fallback to old UIEB logic if config fails? Or just error out?
            # Forcing error is safer:
            raise FileNotFoundError(f"Could not read/parse paths from {config_path}")


        print("--- Dataloader Init ---")
        print(f"Using Input Path: {input_path}")
        print(f"Using GT Path: {gt_path}")
        # ---------------------------

        self.filesA = sorted(glob.glob(input_path + "/*.*"))
        self.filesB = sorted(glob.glob(gt_path + "/*.*"))

        print(f"Found {len(self.filesA)} input files.")
        print(f"Found {len(self.filesB)} GT files.")
        # --------------------------

        if not self.filesA or not self.filesB:
             print("Error: Did not find files in one or both directories. Check paths and contents.")
             self.len = 0 # Ensure len is 0 if no files found
        elif len(self.filesA) != len(self.filesB):
            print(f"Warning: Mismatch in number of files! Input: {len(self.filesA)}, GT: {len(self.filesB)}. Using minimum.")
            self.len = min(len(self.filesA), len(self.filesB))
        else:
            self.len = len(self.filesA)
        print(f"Setting dataset length to: {self.len}")
        print("-----------------------")


    def __getitem__(self, index):
        # Add error checking for file reading
        try:
            img_A = Image.open(self.filesA[index % self.len]).convert('RGB') # Ensure RGB
            img_B = Image.open(self.filesB[index % self.len]).convert('RGB') # Ensure RGB
        except Exception as e:
            print(f"Error opening image at index {index}: {e}")
            # Return dummy data or raise error? Dummy data might hide issues.
            # Returning None might cause issues later. Let's return black images.
            dummy_tensor = torch.zeros((3, 256, 256)) # Assuming 256x256 size
            return {"A": dummy_tensor, "B": dummy_tensor}


        # Apply transforms only if images were loaded successfully
        if np.random.random() < 0.5:
            img_A = Image.fromarray(np.array(img_A)[:, ::-1, :], "RGB")
            img_B = Image.fromarray(np.array(img_B)[:, ::-1, :], "RGB")

        try:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)
        except Exception as e:
            print(f"Error transforming image at index {index}: {e}")
            dummy_tensor = torch.zeros((3, 256, 256))
            return {"A": dummy_tensor, "B": dummy_tensor}


        return {"A": img_A, "B": img_B}

    def __len__(self):
        # print(f"Dataset __len__ called, returning: {self.len}") # Optional debug
        return self.len

    # --- get_file_paths function is NO LONGER USED by __init__ ---
    # def get_file_paths(self, root, dataset_name):
    #     if dataset_name=='UIEB':
    #         filesA, filesB = [], []
    #         sub_dirs = ['train']
    #         for sd in sub_dirs:
    #             filesA += sorted(glob.glob(os.path.join(root, sd, 'trainA') + "/*.*"))
    #             filesB += sorted(glob.glob(os.path.join(root, sd, 'trainB') + "/*.*"))
    #     return filesA, filesB

Overwriting utils/data_utils.py


In [ ]:
%%writefile configs/train_MuLA-GAN.yaml
# --- Training Configuration for MuLA-GAN on GoPro ---

# Dataset Info
dataset_path: '/content/GoPro_Split' # Base path for the split data
TRAIN_INPUT: 'train/trainA'      # Relative path to blurry training images
TRAIN_GT: 'train/trainB'         # Relative path to sharp training images
TEST_INPUT: 'test/testA'         # Relative path to blurry test images
TEST_GT: 'test/testB'            # Relative path to sharp test images
dataset_name: "GoPro"           # Name for the dataset

# Image Info
im_width: 256                   # <<< CORRECTED KEY
im_height: 256                  # <<< CORRECTED KEY
chans: 3                        # <<< CORRECTED KEY

# Model Info
MODEL_NAME: 'MuLA_GAN'
GENERATOR: 'mula_gan_g'
DISCRIMINATOR: 'mula_gan_d'

# Training Params
BATCH_SIZE: 8           # Original Batch Size
NUM_EPOCHS: 200         # Total epochs (adjust as needed)
LR_G: 0.0002
LR_D: 0.0002
B1: 0.5
B2: 0.999
WEIGHT_DECAY: 0.0001
LAMBDA_L1: 100
LAMBDA_ADV: 1
LAMBDA_PERCEPTUAL: 10   # Ensure VGG is used in train script if > 0

# Checkpoint/Logging Params
ckpt_interval: 20       # <<< CORRECTED KEY: Save model every 20 epochs
SAMPLE_INTERVAL: 1000   # Save sample validation images every 1000 steps (iterations)
LOG_INTERVAL: 50        # Print loss info every 50 steps
val_interval: 1000      # <<< CORRECTED KEY: Run validation every 1000 steps

# Output Paths
CHECKPOINT_DIR: 'checkpoints'
SAMPLE_DIR: 'samples'
RESULTS_PATH: 'results'      # Where test.py might save output

Overwriting configs/train_MuLA-GAN.yaml


In [ ]:
!python train_MuLA_GAN.py

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
@@@@@@@@@@@@@@
--- Dataloader Init ---
Using Input Path: /content/GoPro_Split/train/trainA
Using GT Path: /content/GoPro_Split/train/trainB
Found 927 input files.
Found 927 GT files.
Setting dataset length to: 927
-----------------------
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total.

In [ ]:
# --- IMPORTANT: ---
# 1. Replace 'GoPro_Split' if your checkpoint folder has a different name.
# 2. Replace 'generator_199.pth' with the actual highest epoch number.

!python test.py \
  --weights_path "checkpoints/GoPro/generator_300.pth" \
  --data_dir "/content/GoPro_Split/test/testA/" \
  --sample_dir "./output_gopro/"

Loaded model from checkpoints/GoPro/generator_300.pth
Tested: /content/GoPro_Split/test/testA/000002.png
Tested: /content/GoPro_Split/test/testA/000014.png
Tested: /content/GoPro_Split/test/testA/000028.png
Tested: /content/GoPro_Split/test/testA/000033.png
Tested: /content/GoPro_Split/test/testA/000037.png
Tested: /content/GoPro_Split/test/testA/000044.png
Tested: /content/GoPro_Split/test/testA/000045.png
Tested: /content/GoPro_Split/test/testA/000064.png
Tested: /content/GoPro_Split/test/testA/000072.png
Tested: /content/GoPro_Split/test/testA/000078.png
Tested: /content/GoPro_Split/test/testA/000083.png
Tested: /content/GoPro_Split/test/testA/000093.png
Tested: /content/GoPro_Split/test/testA/000104.png
Tested: /content/GoPro_Split/test/testA/000115.png
Tested: /content/GoPro_Split/test/testA/000132.png
Tested: /content/GoPro_Split/test/testA/000143.png
Tested: /content/GoPro_Split/test/testA/000153.png
Tested: /content/GoPro_Split/test/testA/000154.png
Tested: /content/GoPro_Split

In [ ]:
%%writefile Evaluation/measure_ssim_psnr.py
"""
# > Script for measuring quantitative performances in terms of
#   - Structural Similarity Metric (SSIM)
#   - Peak Signal to Noise Ratio (PSNR)
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image
from glob import glob
from os.path import join, exists # Added exists
from ntpath import basename
import os
## local libs
# Make sure imqual_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.imqual_utils import getSSIM, getPSNR
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from imqual_utils import getSSIM, getPSNR
    except ImportError as e:
        print(f"Error importing imqual_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


## compares avg ssim and psnr
def SSIMs_PSNRs(gtr_dir, gen_dir, im_res=(256, 256)):
    """
        - gtr_dir contain ground-truths
        - gen_dir contain generated images
    """
    gtr_paths = sorted(glob(join(gtr_dir, "*.*")))
    # Don't sort gen_paths here, look them up based on gtr_paths
    # gen_paths_all = sorted(glob(join(gen_dir, "*.*")))

    print(f"Found {len(gtr_paths)} potential ground truth images in: {gtr_dir}")
    print(f"Looking for corresponding generated images in: {gen_dir}")

    ssims, psnrs = [], []
    processed_count = 0

    if not gtr_paths:
        print(f"Error: No image files found in ground truth directory: {gtr_dir}")
        return np.array([]), np.array([])

    for gtr_path in gtr_paths:
        # Get base filename without extension
        gtr_f_base = basename(gtr_path).split('.')[0]
        # Construct expected generated file path (try common extensions)
        gen_path_expected_png = join(gen_dir, gtr_f_base + ".png")
        gen_path_expected_jpg = join(gen_dir, gtr_f_base + ".jpg")
        gen_path_expected_jpeg = join(gen_dir, gtr_f_base + ".jpeg")

        gen_path = None
        if exists(gen_path_expected_png):
            gen_path = gen_path_expected_png
        elif exists(gen_path_expected_jpg):
            gen_path = gen_path_expected_jpg
        elif exists(gen_path_expected_jpeg):
            gen_path = gen_path_expected_jpeg
        # Add more extensions if needed

        if gen_path: # Check if a corresponding file was found
            processed_count += 1
            # print(f"Processing pair: GT={gtr_path}, GEN={gen_path}") # Debug
            try:
                r_im = Image.open(gtr_path).resize(im_res)
                g_im = Image.open(gen_path).resize(im_res)

                # Ensure images are RGB before SSIM
                if r_im.mode != 'RGB': r_im = r_im.convert('RGB')
                if g_im.mode != 'RGB': g_im = g_im.convert('RGB')

                # get ssim on RGB channels
                ssim_val = getSSIM(np.array(r_im), np.array(g_im))
                if np.isfinite(ssim_val): ssims.append(ssim_val)

                # get psnr on L channel (grayscale)
                r_im_L = r_im.convert("L")
                g_im_L = g_im.convert("L")
                psnr_val = getPSNR(np.array(r_im_L), np.array(g_im_L))
                if np.isfinite(psnr_val): psnrs.append(psnr_val)

            except Exception as e:
                 print(f"Error processing {basename(gtr_path)} / {basename(gen_path)}: {e}")
        # else:
            # print(f"Skipping {basename(gtr_path)}: Corresponding generated file not found.") # Debug


    if processed_count == 0:
        print("\nError: No matching image pairs found between the two directories!")
        print("Please check paths and ensure filenames (without extension) are identical.")
        return np.array([]), np.array([]) # Return empty arrays

    return np.array(ssims), np.array(psnrs)


# --- Define YOUR paths here ---
# These paths point to your GoPro split data
gtr_dir = "/content/GoPro_Split/test/testB/"  # <<< UPDATED: Path to GoPro test ground truth
gen_dir = "./output_gopro/"               # <<< UPDATED: Path to your generated GoPro test images
# -----------------------------


### compute SSIM and PSNR
SSIM_measures, PSNR_measures = SSIMs_PSNRs(gtr_dir, gen_dir)

# Check if any valid results were returned before calculating mean/std
if len(SSIM_measures) > 0:
    print ("\n--- Results ---")
    print ("SSIM on {0} matched samples".format(len(SSIM_measures)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(SSIM_measures), np.std(SSIM_measures)))
else:
    print("\n--- No valid SSIM scores calculated. ---")


if len(PSNR_measures) > 0:
    print ("\nPSNR on {0} matched samples".format(len(PSNR_measures)))
    print ("Mean: {0:.2f} std: {1:.2f}".format(np.mean(PSNR_measures), np.std(PSNR_measures)))
else:
     print("\n--- No valid PSNR scores calculated. ---")

if len(SSIM_measures) == 0 and len(PSNR_measures) == 0:
    print ("\n--- No results calculated. Check paths, filenames, and potential errors during processing. ---")

Overwriting Evaluation/measure_ssim_psnr.py


In [ ]:
!python Evaluation/measure_ssim_psnr.py

Found 102 potential ground truth images in: /content/GoPro_Split/test/testB/
Looking for corresponding generated images in: ./output_gopro/

--- Results ---
SSIM on 102 matched samples
Mean: 0.8396 std: 0.1349

PSNR on 102 matched samples
Mean: 28.09 std: 4.05


In [ ]:
%%writefile Evaluation/measure_uiqm.py
"""
# > Script for measuring quantitative performance in terms of UIQM
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image, ImageOps
from glob import glob
from os.path import join
from ntpath import basename
import os # Added os import
## local libs
# Make sure uqim_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.uqim_utils import getUIQM
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from uqim_utils import getUIQM
    except ImportError as e:
        print(f"Error importing uqim_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


def measure_UIQMs(dir_name, im_res=(256, 256)):
    paths = sorted(glob(join(dir_name, "*.*")))
    print(f"Found {len(paths)} files in: {dir_name}") # Debug print
    uqims = []
    i=0
    if not paths:
        print(f"Error: No images found in directory: {dir_name}")
        return np.array([])

    for img_path in paths:
        # print(f"Processing image {i+1}/{len(paths)}: {basename(img_path)}") # Debug print
        i=i+1
        try:
            im = Image.open(img_path).resize(im_res)
             # Ensure image is RGB for UIQM calculation
            if im.mode != 'RGB': im = im.convert('RGB')
            im_array = np.array(im)
            # print(im_array.shape) # Debug print shape
            uiqm = getUIQM(im_array)
            if np.isfinite(uiqm): # Check for NaN/Inf
                 uqims.append(uiqm)
            # else:
                # print(f"  -> Warning: UIQM calculation resulted in NaN/Inf for {basename(img_path)}")

        except Exception as e:
             print(f"Error processing {basename(img_path)}: {e}")

    if not uqims:
        print("\nError: UIQM calculation failed for all images.")
        return np.array([])

    return np.array(uqims)


# --- Define YOUR path here ---
# This path points to your generated GoPro results
gen_dir = "./output_gopro/" # <<< UPDATED
# -----------------------------

# UIQMs of the enhanceded output images
gen_uqims = measure_UIQMs(gen_dir)

if len(gen_uqims) > 0: # Only print if results were calculated
    print ("\n--- Results ---")
    print ("UIQM on {0} samples".format(len(gen_uqims)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(gen_uqims), np.std(gen_uqims)))
else:
     print ("\n--- No UIQM results calculated. Check path and logs. ---")

Overwriting Evaluation/measure_uiqm.py


In [ ]:
!python Evaluation/measure_uiqm.py

Found 102 files in: ./output_gopro/

--- Results ---
UIQM on 102 samples
Mean: 3.2567 std: 0.2321


In [ ]:
!zip -r gopro_results.zip ./output_gopro/

  adding: output_gopro/ (stored 0%)
  adding: output_gopro/002147.png (deflated 0%)
  adding: output_gopro/000577.png (deflated 0%)
  adding: output_gopro/002812.png (deflated 0%)
  adding: output_gopro/000681.png (deflated 0%)
  adding: output_gopro/001044.png (deflated 0%)
  adding: output_gopro/001027.png (deflated 0%)
  adding: output_gopro/001386.png (deflated 0%)
  adding: output_gopro/000143.png (deflated 0%)
  adding: output_gopro/002838.png (deflated 0%)
  adding: output_gopro/002486.png (deflated 0%)
  adding: output_gopro/000002.png (deflated 0%)
  adding: output_gopro/000115.png (deflated 0%)
  adding: output_gopro/000212.png (deflated 0%)
  adding: output_gopro/002853.png (deflated 0%)
  adding: output_gopro/002855.png (deflated 0%)
  adding: output_gopro/000270.png (deflated 0%)
  adding: output_gopro/000037.png (deflated 0%)
  adding: output_gopro/000044.png (deflated 0%)
  adding: output_gopro/000248.png (deflated 0%)
  adding: output_gopro/002113.png (deflated 0%)
  ad

In [ ]:
from google.colab import files
files.download('gopro_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>